# Establishing a Database Connection

A series of steps illustrating how to build a database, establish a database connection, test the database connection and pull data utilizing Pisces.

## Building a Database

Data in miniseed or sac formats can be loaded into a sqlite database using commands from pisces. In this tutorial, we will build a sqlite database from a series of SAC files contained in tutorial/cli

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
! pisces sac2db -h

In [ ]:
!pisces sac2db sqlite:///test.sqlite ../infrapy-data/cli/*.SAC

As infrapy is an array processing tool, after your sqlite database is created, you will need to update the REFSTA for each array using update_refsta.py

In [ ]:
%run ../scripts/update_refsta.py sqlite:///test.sqlite FSU
%run ../scripts/update_refsta.py sqlite:///test.sqlite HWU
%run ../scripts/update_refsta.py sqlite:///test.sqlite LCM
%run ../scripts/update_refsta.py sqlite:///test.sqlite PSU
%run ../scripts/update_refsta.py sqlite:///test.sqlite WMU

## Querying the Database

We can now connect to our database and practice querying

In [ ]:
import pisces as ps

session = ps.db_connect(backend='sqlite', instance='test.sqlite')

In [ ]:
session

The tables used in this tutorial are all standard CSS3.0 tables, we define these tables in order to query the database we have created.

In [ ]:
from pisces.tables.css3 import Site, Wfdisc, Origin

In [ ]:
q = session.query(Site).all()

In [ ]:
q

In [ ]:
refsta = session.query(Site).filter(Site.refsta == 'FSU').all()

In [ ]:
refsta

### Retrieve and plot waveforms from a query

In [ ]:
from pisces import wfdisc2trace, read_waveform

In [ ]:
%matplotlib notebook
for wf in session.query(Wfdisc).filter(Wfdisc.chan == 'EDF').limit(4):
    tr = wfdisc2trace(wf)
    data = read_waveform(wf.dir + '/' + wf.dfile, wf.datatype, wf.foff, wf.nsamp)
    tr.plot()